# Association between covariates and PCs of gene expression data

Here we perform a PCA on multi-tissue gene expression data from GTEx v6. Phenotype and RNA-seq gene expression data were downloaded from dbGaP.

**Input data**

* Gene expression profile: See `01_Data_preprocessing.ipynb`.
* Cohort/experimental variable list: All cohort/experiment-related variables including,
  * Technical variables, e.g., RNA-Seq QC metrics (e.g., mapping rates, ...), ...
* Metadata for cohort/experimental variable: data types, ...

**Preparation**

Check input variables:

* Drop variables with too many missing values (e.g., missing values >= 5%).
* Set proper dtype for categorical variables, e.g., gender, batch, ..., etc.

**Global MI between variables**

1. Compute pair-wise MI matrix for all qualified variables using all samples.
2. Output the matrix.

**Tissue-specific covariates**

For a given tissue:

1. Perform PCA on its expression profile, get top $k$ PCs.
2. Compute MI matrix between $k$ PCs and $n$ covariates using normalized mutual information.
3. Output the matrix.

**Notes**

* log2-RPKM performed better than RPKM.

*References*:

* http://scikit-learn.org/stable/auto_examples/feature_selection/plot_f_test_vs_mi.html
* http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html
* http://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection

In [876]:
import numpy as np
import pandas as pd
from omics.stats.PCA import run_pca
from omics.stats.MI import normalized_MI_matrix, MI

%reload_ext version_information
%version_information numpy, pandas, matplotlib, seaborn, omics, sklearn

Software versions
Python 2.7.12 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 5.1.0
OS Linux 2.6.32 431.3.1.el6.x86_64 x86_64 with centos 6.8 Final
numpy 1.11.2
pandas 0.19.1
matplotlib 1.5.3
seaborn 0.7.1
omics 0.1.3
sklearn 0.18.1
Fri Dec 09 02:48:58 2016 EST

# Setup

In [184]:
# Input files from dbGaP
EXPRESSION = "/dev/shm/GTEx/GTEx_Data_20150112_RNAseq_RNASeQCv1.1.8_gene_rpkm.gct"
PHENOTYPE1 = "../../../src/GTEx/dbGaP/phs000424.v6.pht002742.v6.p1.c1.GTEx_Subject_Phenotypes.GRU.txt"
PHENOTYPE2 = "../../../src/GTEx/dbGaP/phs000424.v6.pht002743.v6.p1.c1.GTEx_Sample_Attributes.GRU.txt"

# Data to use in this study
SAMPLE_LIST = "../data/GTEx.samples_to_use.tsv"
GENE_LIST   = "../data/GTEx.genes_to_use.tsv"
SUBJ_VAR_LIST = "../../../src/GTEx/meta/subj_var_list.selected.tsv"
SAMP_VAR_LIST = "../../../src/GTEx/meta/samp_var_list.selected.tsv"

# Ouptut files
OUTPUT_PDATA = "../data/pData.pickle"  # parsed pData for later use
MAPPED_PCA = "../data/samples_x_mapped_PCs.tsv"  # mapped PC as sample features in tissue-specific PCA

GLOBAL_MI  = "../reports/Global_MI_btw_covariates.unnormalized.tsv"
GLOBAL_nMI = "../reports/Global_MI_btw_covariates.tsv"

TISSUE_nMI = "../reports/Tissue-specific_MI_between_PCs_and_covariates.xlsx"
TISSUE_PCA = "../reports/Tissue-specific_PCA_explained_variance_ratios.tsv"

# Others
RND_SEED = 161126  # for calculating MI

# Load input data

## Load expression data

In [158]:
samples = pd.read_table(SAMPLE_LIST, index_col=0, squeeze=True)  # a series (SAMPID -> SMTS)
genes = pd.read_table(GENE_LIST, index_col=0, squeeze=True)  # a series (ENSG ID -> Description)

print len(samples), 'samples.'
print len(genes), 'genes.'

8525 samples.
21146 genes.


In [45]:
exprs = pd.read_table(EXPRESSION, header=2, index_col='Name', usecols=['Name'] + samples.index.tolist()).loc[genes.index]
exprs = np.log2(exprs + 1)  # log2-transformed after adding a pseudo-count 1
exprs.shape

(21146, 8525)

In [46]:
# Test
assert 'GTEX-1117F-0226-SM-5GZZ7' not in exprs  # not in data freeze due to Poor D-statistic
assert 'GTEX-111CU-1826-SM-5GZYN' in exprs

## Load phenotype data

Date variables, no matter in Date format (ordinal) or String format (categorical), yield the same MI as long as they treated as discrete variables.

In [139]:
subj_var = pd.read_table(SUBJ_VAR_LIST, index_col=0)  # a dataframe (name -> description, type)
samp_var = pd.read_table(SAMP_VAR_LIST, index_col=0)  # a dataframe (name -> description, type)
meta_var = pd.concat([subj_var, samp_var])

print subj_var.shape[0], "subject variables."
print samp_var.shape[0], "sample variables."

146 subject variables.
62 sample variables.


Join subject phenotype & sample attribute variables. (Note that 'SAMPID' variable is used as index)

In [140]:
pData1 = pd.read_table(PHENOTYPE1, index_col="SUBJID", skiprows=10, usecols=subj_var.index)
pData2 = pd.read_table(PHENOTYPE2, index_col="SAMPID", skiprows=10, usecols=samp_var.index, \
                       parse_dates=['SMNABTCHD', 'SMGEBTCHD']).loc[samples.index]

pData2['SUBJID'] = ['-'.join(i.split('-')[:2]) for i in pData2.index if i.startswith('GTEX')]  # get SUBJID from SAMPID
pData = pData2.join(pData1, on="SUBJID")  # join subject variables to sample variable dataframe
print '(#Samples, #Variables):', pData.shape

(#Samples, #Variables): (8525, 207)


## Set categorical variables

External variable type metadata -> Internal pandas dtypes, e.g., there are 4 types of variables in GTEx metadata:

* `integer, encoded value`
* `integer`
* `decimal`
* `string`

Among them, `integer, encoded value` and `string` should be encoded as "category" in general.

There could be some exceptions, e.g., `SMATSSCR` (Autolysis Score) is integer-encoded, but it is an ordinal category variable. In such case, no need to re-type them for Kendall's tau and tree-based algorithms. For mutual information, it makes no difference between ordinal or non-ordinal category variables, just make it as categorical then you are fine.

Date variables are tricky. They should be encoded as date object so that the date order will be meaningful. However, if they were used for "Batch date", then the order of date doesn't really make any sense (close dates do not necessary mean the batches were "close"). In such cases, they should stay as string and be encoded as categorical variables.

In [141]:
# Here we define a function that read pData and variable metadata to assign correct categorical dtype for each categorical variable.
def set_categorical_dtype(pData, metadata):
    """Set categorical columns to categorical dtype.
    
    pData: pData dataframe.
    metadata: variable metadata, including variable type.
    
    No return. Fix in place.
    """
    for i,x in pData.iteritems():
        if metadata.type[i] in ['string', 'integer, encoded value']:
            pData[i] = x.astype('category')

In [142]:
set_categorical_dtype(pData, meta_var)
print 'Now we have', pData.shape[1], 'variables:'
print pData.dtypes.value_counts()

Now we have 207 variables:
category    151
float64      56
dtype: int64


In [185]:
pData.to_pickle(OUTPUT_PDATA)  # save for later use

## Clean variables

Here to discard those variables with too many missing values and to decide what to do with left missing values. We need following functions:

1. `clean_variables(pData)`: Remove unqualified varialbes (too many missing values or invariant).
2. `impute with mean for continous variables`: No need for categorical because pandas category automatically encodes nan as -1.

In [145]:
def impute_pData(pData):
    return pData.fillna(pData.mean(numeric_only=True))

def clean_pData(pData, cutoff=0.1):
    n, k = pData.shape  # n samples x k variables
    
    is_enough = lambda x: len(x.dropna()) > n * (1-cutoff)  # %missing values < cutoff
    is_stateful = lambda x: n > len(set(x.dropna())) > 1  # no. states should be > 1 and < n
    is_informative = lambda x: MI(x, x) > cutoff  # information content (entropy) > cutoff
    is_variant = lambda x: np.std(x) > 0
    
    pass1 = [i for i,x in pData.select_dtypes(['category']).iteritems() if is_enough(x) and is_stateful(x) and is_informative(x)]
    pass2 = [i for i,x in pData.select_dtypes([np.number]).iteritems()  if is_enough(x) and is_variant(x)]
    print k, 'variables ->', len(pass1), 'categorical variables and', len(pass2), 'numeric variables.'
        
    return pData[sorted(pass1 + pass2)]

# Global MI between variables

Here we calculate pair-wise normalized mutual information for all variables using all samples. We will be able to see the global inter-dependency structure of covariates.

Ref:
* https://en.wikipedia.org/wiki/Mutual_information#Normalized_variants

In [865]:
mat, mat2 = normalized_MI_matrix(impute_pData(clean_pData(pData)), seed=RND_SEED, verbose=True)

mat.to_csv(GLOBAL_MI, sep="\t")  # MI matrix
mat2.to_csv(GLOBAL_nMI, sep="\t")  # normalized MI matrix

Input dataframe: 8525 samples x 146 features
95 features are categorical
Computing pair-wise MI ...
Normalize MI ...


# Tissue-specific covariates

In [148]:
def pc_vs_covariates_MI(exprs, pData, npc=10):
    # Run PCA
    pca, exprs_new = run_pca(exprs.T, pc=npc)
    print "Total explained variance of top %d PCs: %.2f" % (npc, pca.explained_variance_ratio_.sum())
    
    # Joined feature dataframe (PCs + variables)
    assert all(exprs_new.index == pData.index)  # check data integrity
    df = pd.concat([exprs_new, pData], axis=1)
    
    # Compute normalized MI matrix
    mat, nmat = normalized_MI_matrix(df, seed=RND_SEED, verbose=True)  # MI, nMI

    return pca, exprs_new, mat, nmat


def run_all_tissues(exprs, samples, pData, TISSUE_nMI, TISSUE_PCA, MAPPED_PCA):
    xlsx = pd.ExcelWriter(TISSUE_nMI)  # output: tissue-specific MI matrix as a multi-sheet excel file
    pca_var = dict()  # explained variance ratio of top n PCs in each tissue
    pc_dfs = []  # mapped PC dataframes

    for tissue in samples.astype('category').cat.categories:
        sub_samples = samples[samples == tissue].index
        sub_exprs = exprs[sub_samples]
        sub_pData = pData.loc[sub_samples]
        sub_pData2 = impute_pData(clean_pData(sub_pData))

        print '================================'
        print tissue
        print '================================'
        print 'Expression data:', sub_exprs.shape
        print 'Phenotype data:', sub_pData.shape
        print 'Imputed pData:', sub_pData2.shape

        pca, exprs_new, mat, mat2 = pc_vs_covariates_MI(sub_exprs, sub_pData2)

        pca_var[tissue] = pca.explained_variance_ratio_
        pc_dfs.append(exprs_new)
        mat2.to_excel(xlsx, tissue)
        print

    xlsx.save()
    xlsx.close()
    pd.DataFrame(pca_var).to_csv(TISSUE_PCA2, sep="\t")
    pd.concat(pc_dfs).to_csv(MAPPED_PCA, sep="\t")

Iterate all tissues:

In [147]:
run_all_tissues(exprs, samples, pData, TISSUE_nMI, TISSUE_PCA, MAPPED_PCA)

207 variables -> 98 categorical variables and 53 numeric variables.
Adipose Tissue
Expression data: (21146, 577)
Phenotype data: (577, 207)
Imputed pData: (577, 151)
Data dimensions (samples-by-features): (577, 21146)
Variance explained by top 10 PCs: [ 0.15892685  0.14077131  0.06445746  0.0487673   0.038848    0.03683041
  0.0302921   0.02412104  0.02005552  0.01581516]
Total explained variance of top 10 PCs: 0.58
Input dataframe: 577 samples x 161 features
98 features are categorical
Computing pair-wise MI ...
Normalize MI ...

207 variables -> 88 categorical variables and 55 numeric variables.
Adrenal Gland
Expression data: (21146, 145)
Phenotype data: (145, 207)
Imputed pData: (145, 143)
Data dimensions (samples-by-features): (145, 21146)
Variance explained by top 10 PCs: [ 0.15831179  0.09821024  0.0833971   0.05271907  0.03567548  0.03123482
  0.02979179  0.02467147  0.02407093  0.01935258]
Total explained variance of top 10 PCs: 0.56
Input dataframe: 145 samples x 153 features
